In [ ]:
import comet_ml
from comet_ml import Experiment
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
%matplotlib inline

In [ ]:
samples=1000
step=1e-7
path='../data/'+str(samples)+'_'+str(step)+'/'
i=1


In [ ]:
import pandas as pd
Input=pd.read_csv(path+'input/input_'+str(i)+'.csv', sep=',')
input_var=Input.columns[:8]
Input=torch.from_numpy(Input[input_var].values)

Output=pd.read_csv(path+'output/output_'+str(i)+'.csv', sep=',')
Output_var=Output.columns[1:8]
Output=torch.from_numpy(Output[Output_var].values)

In [ ]:
   
# Add the following code anywhere in your machine learning file
experiment = Experiment(api_key="D4ClOaV8GaZLHo0I2et7bOHgy",
                        project_name="ann-combustion-test", workspace="mdcabrera")
